The code block below makes sure that you get some information which you can use for debugging, cleans up the output from unecessary warnings and defines a function which whill print json outputs in a nicely formatted way.

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

In [2]:
import rasa_nlu
import sklearn
import spacy
import rasa_core

example : Conversation_1:
U: Hello.
B: Hey! I am ThingWorx assistant. How can I help?
U: what are the machines available
B: Forklift001,Forklift002,Forklift003,Forklift004,Forklift005,Forklift006
U: show me about forklift001
B: to see the status click here : http://www.iotspace.io/Thingworx/Things/Forklift001/Properties
U: Thanks.
B: Happy reading.
U: Goodbye.
B: Goodbye.

# Creating the training data for language understanding model
Lets create some training data here by grouping user messages by their intents. The intent describes what the messages mean. Another important part of training data are entities - pieces of information which help a chatbot understand what specifically a user is asking about. Entities are labeled using the markdown link syntex: [entity value](entity_type)

In [3]:
nlu_md = """

## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really
- nope
- definitely no
- no no


## intent:machine_status
- what are the machines available
- list the machines

## intent:machine_name
- show me about [forklift001](machine_type)
- [forklift002](machine_type)
- about [forklift002](machine_type)
- maybe [forklift003](machine_type)
- about [forklift004](machine_type)
- for [forklift005](machine_type)
- about [forklift006](machine_type)
- maybe about [forklift001](machine_type)

## intent:machine_status+machine_name
- show me the machine status of [forklift002](machine_type)
- give me the details of [forklift003](machine_type)


## intent:thanks
- thanks
- thank you
- thank you very much
- thanks a lot
- thank you so much
- thank you loads
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [4]:
config = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"                              #defines how unstructured sentences will be tokenized
- name: "ner_crf"                                           #defines the model which will be used for entity extraction
- name: "intent_featurizer_count_vectors"                   #creates sentence representation
- name: "intent_classifier_tensorflow_embedding"            #defines a classifier for intent classification
  intent_tokenization_flag: true                            #sets the flag for intent label tokenization
  intent_split_symbol: "+"                                  #defines the character on which intent labels should be tokenized
""" 

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


# Training the Rasa NLU Model


In [5]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data, verbose=True)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")


INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 51 (7 distinct intents)
	- Found intents: 'machine_name', 'machine_status', 'machine_status+machine_name', 'goodbye', 'deny', 'greet', 'thanks'
	- entity examples: 10 (1 distinct entities)
	- found entities: 'machine_type'

INFO:rasa_nlu.model:Starting to train component WhitespaceTokenizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component CRFEntityExtractor
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component CountVectorsFeaturizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component EmbeddingIntentClassifier



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:rasa_nlu.classifiers.embedding_intent_classifier:Accuracy is updated every 10 epochs
Epochs: 100%|████████████████████████████████████████████████| 300/300 [00:02<00:00, 136.95it/s, loss=0.105, acc=0.980]
INFO:rasa_nlu.classifiers.embedding_intent_classifier:Finished training embedding classifier, loss=0.105, train accuracy=0.980
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Successfully saved model into 'C:\Users\sreeram\Desktop\chatbot\models\nlu\default\current'


# Writing Stories


In [6]:
stories_md = """


## Suggestion path 1
* greet
  - utter_greet
* machine_status
  - utter_list
* machine_name{"machine_type":"forklift001"}
  - action_machine_link
  - utter_link
* goodbye
  - utter_goodbye

## Suggestion path 2
* greet
  - utter_greet
* machine_status+machine_name{"machine_type":"forklift001"}
  - action_machine_link
  - utter_link
* thanks
  - utter_welcome
* goodbye
  - utter_goodbye

## Suggestion path 3
* greet
  - utter_greet
* machine_name{"machine_type":"forklift001"}
  - action_machine_link
  - utter_link
* deny
  - utter_goodbye

"""

%store stories_md > stories.md

print("Done!")

Writing 'stories_md' (str) to file 'stories.md'.
Done!


# Domain

In [7]:
domain_yml = """
intents:
- greet
- goodbye
- thanks
- deny
- machine_status+machine_name
- machine_status
- machine_name
slots:
  machine_type:
    type: text
    
entities:
- paper_type

actions:
- utter_greet
- utter_goodbye
- utter_link
- utter_welcome
- utter_list
- action_machine_link

templates:
  utter_greet:
  - text: "Hey! I am ThingWorx assistant. How can I help"

  utter_goodbye:
  - text: "Have a great day!"

  utter_link:
  - text: "to see the status click here : {machine_type}"
  
  utter_welcome:
  - text: "It was pleasure serving you"
  
  utter_list:
  - text: "Forklift001,Forklift002,Forklift003,Forklift004,Forklift005,Forklift006"
"""

%store domain_yml > domain.yml


Writing 'domain_yml' (str) to file 'domain.yml'.


In [8]:
custom_action = """

from rasa_sdk import Action
from rasa_sdk.events import SlotSet

import requests

class action_machine_link(Action):
    def name(self):
        return "action_machine_link"

    def run(self, dispatcher, tracker, domain):

        machine_type = tracker.get_slot('machine_type')
        link=""
        if('f' in machine_type):
            machine_type=list(machine_type)
            machine_type[0]='F'
            machine_type="".join(machine_type)
            link="http://www.iotspace.io/Thingworx/Things/{}/Properties?apikey=e82d0e47-9f2a-42c6-b6e4-a9006cdf9b18".format(machine_type)
        else:
             dispatcher.utter_message("wrong machine_type")
        dispatcher.utter_message(machine_type)
        return [SlotSet("machine_type",link)]
        
"""
%store custom_action > actions.py

Writing 'custom_action' (str) to file 'actions.py'.


# Training your Dialogue Model


Now we are going to train the dialogue management model. We can specify what policies should be used to train it - in this case, the model is a neural network implemented in Keras which learns to predict which action to take next. We can also change the parameters of what percentage of training examples should be used for validation and how many epochs should be used for training.

In [9]:
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.agent import Agent

agent = Agent('domain.yml', policies=[KerasPolicy(
        validation_split=0.0,
        epochs=250)])
training_data = agent.load_data('stories.md')
agent.train(
        training_data
)

agent.persist('models/dialogue')


INFO:apscheduler.scheduler:Scheduler started
Processed Story Blocks: 100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 208.57it/s, # trackers=3]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 23)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7168      
_________________________________________________________________
dense (Dense)                (None, 14)                462       
_________________________________________________________________
activation (Activation)      (None, 14)                0         
Total params: 7,630
Trainable params: 7,630
Non-trainable params: 0
_________________________________________________________________


INFO:rasa_core.policies.keras_policy:Fitting model with 48 total samples and a validation split of 0.0


Epoch 1/250
48/48 [==============================] - ETA: 0s - loss: 2.6008 - acc: 0.093 - 1s 13ms/sample - loss: 2.6017 - acc: 0.0625
Epoch 2/250
48/48 [==============================] - ETA: 0s - loss: 2.5536 - acc: 0.156 - 0s 320us/sample - loss: 2.5478 - acc: 0.1667
Epoch 3/250
48/48 [==============================] - ETA: 0s - loss: 2.5120 - acc: 0.312 - 0s 414us/sample - loss: 2.5121 - acc: 0.3125
Epoch 4/250
48/48 [==============================] - ETA: 0s - loss: 2.4904 - acc: 0.312 - 0s 362us/sample - loss: 2.4943 - acc: 0.2917
Epoch 5/250
48/48 [==============================] - ETA: 0s - loss: 2.4782 - acc: 0.281 - 0s 382us/sample - loss: 2.4686 - acc: 0.2917
Epoch 6/250
48/48 [==============================] - ETA: 0s - loss: 2.4491 - acc: 0.281 - 0s 382us/sample - loss: 2.4323 - acc: 0.3125
Epoch 7/250
48/48 [==============================] - ETA: 0s - loss: 2.4054 - acc: 0.375 - 0s 393us/sample - loss: 2.4013 - acc: 0.4375
Epoch 8/250
48/48 [==============================

Epoch 61/250
48/48 [==============================] - ETA: 0s - loss: 1.5383 - acc: 0.375 - 0s 279us/sample - loss: 1.4421 - acc: 0.4375
Epoch 62/250
48/48 [==============================] - ETA: 0s - loss: 1.5191 - acc: 0.406 - 0s 320us/sample - loss: 1.4094 - acc: 0.4792
Epoch 63/250
48/48 [==============================] - ETA: 0s - loss: 1.4993 - acc: 0.375 - 0s 310us/sample - loss: 1.3941 - acc: 0.4583
Epoch 64/250
48/48 [==============================] - ETA: 0s - loss: 1.4908 - acc: 0.406 - 0s 300us/sample - loss: 1.3700 - acc: 0.4792
Epoch 65/250
48/48 [==============================] - ETA: 0s - loss: 1.5547 - acc: 0.406 - 0s 289us/sample - loss: 1.4192 - acc: 0.4792
Epoch 66/250
48/48 [==============================] - ETA: 0s - loss: 1.4551 - acc: 0.468 - 0s 310us/sample - loss: 1.3451 - acc: 0.5208
Epoch 67/250
48/48 [==============================] - ETA: 0s - loss: 1.4713 - acc: 0.437 - 0s 331us/sample - loss: 1.3423 - acc: 0.5208
Epoch 68/250
48/48 [=====================

48/48 [==============================] - ETA: 0s - loss: 0.9215 - acc: 0.718 - 0s 351us/sample - loss: 0.8273 - acc: 0.7917
Epoch 121/250
48/48 [==============================] - ETA: 0s - loss: 0.9119 - acc: 0.781 - 0s 269us/sample - loss: 0.8156 - acc: 0.8333
Epoch 122/250
48/48 [==============================] - ETA: 0s - loss: 0.8893 - acc: 0.843 - 0s 351us/sample - loss: 0.8013 - acc: 0.8750
Epoch 123/250
48/48 [==============================] - ETA: 0s - loss: 0.8927 - acc: 0.812 - 0s 362us/sample - loss: 0.7879 - acc: 0.8542
Epoch 124/250
48/48 [==============================] - ETA: 0s - loss: 0.8419 - acc: 0.875 - 0s 341us/sample - loss: 0.7561 - acc: 0.8958
Epoch 125/250
48/48 [==============================] - ETA: 0s - loss: 0.8954 - acc: 0.812 - 0s 341us/sample - loss: 0.7881 - acc: 0.8333
Epoch 126/250
48/48 [==============================] - ETA: 0s - loss: 0.8278 - acc: 0.875 - 0s 300us/sample - loss: 0.7459 - acc: 0.8958
Epoch 127/250
48/48 [===========================

48/48 [==============================] - ETA: 0s - loss: 0.5034 - acc: 0.906 - 0s 310us/sample - loss: 0.4342 - acc: 0.9375
Epoch 180/250
48/48 [==============================] - ETA: 0s - loss: 0.5255 - acc: 0.843 - 0s 331us/sample - loss: 0.4518 - acc: 0.8958
Epoch 181/250
48/48 [==============================] - ETA: 0s - loss: 0.4808 - acc: 0.875 - 0s 269us/sample - loss: 0.4207 - acc: 0.9167
Epoch 182/250
48/48 [==============================] - ETA: 0s - loss: 0.4841 - acc: 0.875 - 0s 300us/sample - loss: 0.4254 - acc: 0.9167
Epoch 183/250
48/48 [==============================] - ETA: 0s - loss: 0.4749 - acc: 0.875 - 0s 279us/sample - loss: 0.4127 - acc: 0.9167
Epoch 184/250
48/48 [==============================] - ETA: 0s - loss: 0.4773 - acc: 0.906 - 0s 331us/sample - loss: 0.4141 - acc: 0.9375
Epoch 185/250
48/48 [==============================] - ETA: 0s - loss: 0.5303 - acc: 0.906 - 0s 331us/sample - loss: 0.4648 - acc: 0.9167
Epoch 186/250
48/48 [===========================

48/48 [==============================] - ETA: 0s - loss: 0.2844 - acc: 0.906 - 0s 310us/sample - loss: 0.2499 - acc: 0.9167
Epoch 239/250
48/48 [==============================] - ETA: 0s - loss: 0.2591 - acc: 0.937 - 0s 279us/sample - loss: 0.2283 - acc: 0.9583
Epoch 240/250
48/48 [==============================] - ETA: 0s - loss: 0.3247 - acc: 0.875 - 0s 320us/sample - loss: 0.2648 - acc: 0.9167
Epoch 241/250
48/48 [==============================] - ETA: 0s - loss: 0.3910 - acc: 0.843 - 0s 320us/sample - loss: 0.3160 - acc: 0.8958
Epoch 242/250
48/48 [==============================] - ETA: 0s - loss: 0.2448 - acc: 0.968 - 0s 300us/sample - loss: 0.2074 - acc: 0.9792
Epoch 243/250
48/48 [==============================] - ETA: 0s - loss: 0.2256 - acc: 0.937 - 0s 300us/sample - loss: 0.2048 - acc: 0.9583
Epoch 244/250
48/48 [==============================] - ETA: 0s - loss: 0.2796 - acc: 0.937 - 0s 320us/sample - loss: 0.2410 - acc: 0.9583
Epoch 245/250
48/48 [===========================

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory models/dialogue exists and contains old model files. All files will be overwritten.
INFO:rasa_core.agent:Persisted model to 'C:\Users\sreeram\Desktop\chatbot\models\dialogue'


In [10]:
endpoint = """
action_endpoint:
  url: "http://localhost:5055/webhook"
"""
%store endpoint > endpoints.yml

Writing 'endpoint' (str) to file 'endpoints.yml'.


In [11]:
from rasa_core.actions import Action
from IPython.display import IFrame
IFrame("http://localhost:8888/terminals/1", width=900, height=500)
#next run this python -m rasa_core_sdk.endpoint --actions actions C:\ProgramData\Anaconda3\Lib\site-packages

In [12]:
import IPython
from IPython.display import clear_output
from rasa_core.agent import Agent
from rasa_core.interpreter import NaturalLanguageInterpreter
from rasa_core.utils import EndpointConfig
import time

messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
interpreter = NaturalLanguageInterpreter.create('C:/Users/sreeram/Desktop/chatbot/models/nlu/default/current')
endpoint = EndpointConfig('http://localhost:5055/webhook')
agent = Agent.load('models/dialogue', interpreter=interpreter, action_endpoint = endpoint)

print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input().lower()
    if a == 'stop':
        break
    responses = agent.handle_text(a)
    for response in responses:
        print(response["text"])

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:/Users/sreeram/Desktop/chatbot/models/nlu/default/current\component_3_EmbeddingIntentClassifier.ckpt


Your bot is ready to talk! Type your messages here or send 'stop'
hi
Hey! I am ThingWorx assistant. How can I help
show me the machine status of forklift001
Forklift001
to see the status click here : http://www.iotspace.io/Thingworx/Things/Forklift001/Properties?apikey=e82d0e47-9f2a-42c6-b6e4-a9006cdf9b18
thank you
It was pleasure serving you
stop


In [13]:
pprint(interpreter.parse("forklift001"))


{
  "intent": {
    "name": "machine_name",
    "confidence": 0.6929658651351929
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "machine_name",
      "confidence": 0.6929658651351929
    },
    {
      "name": "greet",
      "confidence": 0.4679192304611206
    },
    {
      "name": "goodbye",
      "confidence": 0.23245006799697876
    },
    {
      "name": "deny",
      "confidence": 0.13156992197036743
    },
    {
      "name": "thanks",
      "confidence": 0.0
    },
    {
      "name": "machine_status+machine_name",
      "confidence": 0.0
    },
    {
      "name": "machine_status",
      "confidence": 0.0
    }
  ],
  "text": "forklift001",
  "model": "current",
  "project": "default"
}
